In [1]:
import pandas as pd
import numpy as np
import flaml
import sklearn.metrics
from flaml import AutoML

In [5]:
data = pd.read_pickle('standardized_pca.pkl')
to_scam = pd.read_pickle('to_scam.pkl')
from_scam = pd.read_pickle('from_scam.pkl')

In [6]:
# get single column indicating if transactions is fraudulent
temp = to_scam + from_scam
scam =[]
for i in temp:
    if i==0:
        scam.append(0)
    else:
        scam.append(1)
target = pd.DataFrame(scam)
target

,0
0,0
1,0
2,0
3,0
4,0
...,...
71245,1
71246,1
71247,1
71248,1


In [14]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-2.851219e+06,-716693.204317,-9.129557,-0.807099,-0.326903,-0.163760,1.291557,-0.570831,-0.774800,1.354555,-0.175839,-0.051796,0.035216,-0.000970,-3.490064e-17
1,2.429850e+06,-945645.225879,-6.382802,-0.073431,0.533238,0.021518,1.251607,-0.494737,-0.865009,1.251950,-0.471838,-0.441421,0.055196,-0.001801,-8.849623e-17
2,-1.612115e+06,-770406.890727,-9.415143,-0.661439,0.510493,0.087232,1.268334,-0.477160,-0.845780,1.218349,-0.412746,-0.050306,0.045578,-0.000991,-7.217999e-17
3,-5.181130e+05,-817835.804779,-8.594401,-0.501390,0.513395,0.090977,1.235920,-0.491985,-0.849519,1.208861,-0.428138,0.210128,0.042545,-0.000569,-4.962858e-17
4,-4.132115e+05,-822383.754038,-8.515702,-0.486031,0.513626,0.091333,1.232373,-0.493584,-0.849879,1.207671,-0.429602,0.240678,0.042168,-0.000519,-4.702650e-17


# SVM Learner

In [68]:
from flaml.model import SKLearnEstimator
from sklearn.linear_model import LogisticRegression

class log_learner(SKLearnEstimator):
    def __init__(self, task="binary"):
        super().__init__(task, **config)
        self.estimator_class = LogisticRegression
        print(config)

# Random Forest

In [69]:
automl = AutoML()
automl.add_learner("log", log_learner)
automl_settings = {
    "metric": 'log_loss',
    "task": 'classification',
    "log_file_name": "flaml_rf_xg.log",
}

In [71]:
automl.fit(X_train=data, y_train=np.array(target),
           **automl_settings,estimator_list=['log','rf','xgboost'],n_splits=5,model_history=True)

[flaml.automl: 03-22 15:44:13] {2055} INFO - task = classification
[flaml.automl: 03-22 15:44:13] {2057} INFO - Data split method: stratified
[flaml.automl: 03-22 15:44:13] {2061} INFO - Evaluation method: holdout
[flaml.automl: 03-22 15:44:13] {2142} INFO - Minimizing error metric: log_loss
[flaml.automl: 03-22 15:44:13] {2200} INFO - List of ML learners in AutoML Run: ['log', 'rf', 'xgboost']
[flaml.automl: 03-22 15:44:13] {2453} INFO - iteration 0, current learner log


TypeError: __init__() got an unexpected keyword argument 'n_jobs'

In [55]:
automl.best_loss_per_estimator

{'rf': 0.1522431472597512, 'xgboost': 0.08177344729835495}

In [ ]:
%